In [1]:
import numpy as np
import torch
import pyepo

# random seed
np.random.seed(42)
torch.manual_seed(42)

Auto-Sklearn cannot be imported.


In [2]:
import cvxpy as cvx
cvx.installed_solvers()

(CVXPY) Nov 08 11:19:56 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')
(CVXPY) Nov 08 11:19:57 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


['CLARABEL', 'COPT', 'ECOS', 'ECOS_BB', 'GUROBI', 'MOSEK', 'SCIPY', 'SCS']

## Data Set and Optimization Solver

In [3]:
# generate data
grid = (5,5) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [4]:
from pyepo.model.grb import shortestPathModel
# set solver
optmodel = shortestPathModel(grid)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 4.5703345390527765
(0, 5)
(5, 6)
(6, 11)
(11, 12)
(12, 17)
(17, 18)
(18, 19)
(19, 24)


In [5]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)

In [6]:
from dataset import optDatasetConstrs
# get training and test data set
dataset_train = optDatasetConstrs(optmodel, x_train, costs=c_train) # with binding constr
dataset_test = pyepo.data.dataset.optDataset(optmodel, x_test, costs=c_test) # without binding constr

Optimizing for optDataset...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 337.96it/s]


Optimizing for optDataset...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 535.24it/s]


In [7]:
# get training and test data set without costs
dataset_train = optDatasetConstrs(optmodel, x_train, sols=dataset_train.sols) # with binding constr

Obtaining constraints for optDataset...


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 493.23it/s]


In [8]:
# get data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Prediction Model

In [9]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, (grid[0]-1)*grid[1]+(grid[1]-1)*grid[0])

    def forward(self, x):
        out = self.linear(x)
        return out

## Train

In [10]:
import time
def train(reg, ca_cos, lr, num_epochs, log_step):
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # init log
    loss_log, regret_log = [], [pyepo.metric.regret(reg, optmodel, loader_test)]
    # running time
    elapsed = 0
    for epoch in range(num_epochs):
        tick = time.time()
        for data in loader_train:
            x, w, t_ctr = data
            # forward pass
            cp = reg(x)
            loss = ca_cos(cp, t_ctr)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_log.append(loss.item())
        # record time
        tock = time.time()
        elapsed += tock - tick
        if epoch % log_step == 0:
            # regret
            regret = pyepo.metric.regret(reg, optmodel, loader_test)
            regret_log.append(regret)
            print("Epoch {:3}, Loss: {:8.4f}, Regret: {:7.4f}%".format(epoch, loss.item(), regret*100))
    print("Elapsed Time: {:.2f} Sec".format(elapsed))
    return loss_log, regret_log

### Gurobi Method 

In [11]:
import gurobipy as gp
from gurobipy import GRB

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # ceate a model
        m = gp.Model("projection")
        # turn off output
        m.Params.outputFlag = 0
        # varibles
        p = m.addMVar(len(cp), name="x", lb=-GRB.INFINITY)
        λ = m.addMVar(len(ctr), name="λ")
        # onjective function
        obj = (cp - p) @ (cp - p)
        m.setObjective(obj, GRB.MINIMIZE)
        # constraints
        m.addConstr(ctr.T @ λ == p)
        # focus on numeric problem
        m.Params.NumericFocus = 3
        # solve
        m.optimize()
        # get solutions
        proj = np.array(p.X)
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [12]:
# init model
reg = LinearRegression()

In [13]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [14]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

Epoch   0, Loss:  -0.9972, Regret: 28.5068%
Elapsed Time: 6.61 Sec


## Gurobi with CVXPY

In [15]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # varibles
        p = cvx.Variable(len(cp), name="x")
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - p))
        # constraints
        constraints = [ctr.T @ λ == p]
        # ceate a model
        problem = cvx.Problem(objective, constraints)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.GUROBI, solver_opts={"NumericFocus": 3})
        # get solutions
        proj = p.value
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [16]:
# init model
reg = LinearRegression()

In [17]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [18]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

Epoch   0, Loss:  -0.9949, Regret: 23.2719%
Elapsed Time: 12.40 Sec


## COPT 

In [19]:
from coptpy import Envr, EnvrConfig
from coptpy import COPT

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # no banner
        envconfig = EnvrConfig()
        envconfig.set('nobanner', '1')
        # ceate a model
        m = Envr(envconfig).createModel("projection")
        # turn off output
        m.setParam("Logging", 0)
        # varibles
        p = m.addMVar(len(cp), nameprefix="x", lb=-COPT.INFINITY)
        λ = m.addMVar(len(ctr), nameprefix="λ")
        # onjective function
        obj = (cp - p) @ (cp - p)
        m.setObjective(obj, COPT.MINIMIZE)
        # constraints
        m.addConstr(ctr.T @ λ == p)
        # focus on numeric problem
        m.setParam("NumericFocus", 3)
        # solve
        m.solve()
        # get solutions
        proj = np.array(p.X)
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [20]:
# init model
reg = LinearRegression()

In [21]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [22]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

##  COPT with CVXPY

In [23]:
import cvxpy as cvx

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # varibles
        p = cvx.Variable(len(cp), name="x")
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - p))
        # constraints
        constraints = [ctr.T @ λ == p]
        # ceate a model
        problem = cvx.Problem(objective, constraints)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.COPT, NumericFocus=3)
        # get solutions
        proj = p.value
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [24]:
# init model
reg = LinearRegression()

In [25]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [26]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

## Mosek with CVXPY 

In [27]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # varibles
        p = cvx.Variable(len(cp), name="x")
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - p))
        # constraints
        constraints = [ctr.T @ λ == p]
        # ceate a model
        problem = cvx.Problem(objective, constraints)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.MOSEK)
        # get solutions
        proj = p.value
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [28]:
# init model
reg = LinearRegression()

In [29]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [30]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

Epoch   0, Loss:  -0.9966, Regret: 22.1378%
Elapsed Time: 15.68 Sec


##  Clarabel

In [31]:
import clarabel as cl
from scipy import sparse

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # number of variable
        num_p = len(cp)
        num_λ = len(ctr)
        # objective function: min ||cp - p||^2 => p^T * I * p - 2 * cp^T * p + cp.T * cp
        # quadratic term
        Q = np.zeros((num_p+num_λ, num_p+num_λ))
        Q[:num_p, :num_p] = np.eye(num_p)
        Q = sparse.csc_matrix(Q)
        # linear term
        p = np.zeros(num_p+num_λ)
        p[:num_p] = - 2 * cp
        # constraints
        A = np.zeros((num_p+num_λ, num_p+num_λ))
        b = sparse.csc_matrix(np.zeros(num_p+num_λ))
        # ctr.T @ λ == p =>  [-I, ctr.T] * [p, λ] == 0
        A[:num_p] = np.hstack([-np.eye(num_p), ctr.T])
        # λ >= 0
        A[num_p:,num_p:] = - np.eye(num_λ)
        # cone
        cones = [cl.ZeroConeT(num_p), cl.NonnegativeConeT(num_λ)]
        # settings
        settings = cl.DefaultSettings()
        settings.verbose = False
        # QP model
        model = cl.DefaultSolver(Q, p, A, b, cones, settings)
        # solve
        result = model.solve()
        # get the solution
        proj = result.x[:num_p]
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [32]:
# init model
reg = LinearRegression()

In [33]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [34]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

NotImplementedError: multi-dimensional sub-views are not implemented

## Clarabel with CVXPY

In [35]:
import cvxpy as cvx

from func import exactConeAlignedCosine

class coneAlignedCosine(exactConeAlignedCosine):
    def _getProjection(self, cp, ctr):
        # varibles
        p = cvx.Variable(len(cp), name="x")
        λ = cvx.Variable(len(ctr), name="λ", nonneg=True)
        # onjective function
        objective = cvx.Minimize(cvx.sum_squares(cp - p))
        # constraints
        constraints = [ctr.T @ λ == p]
        # ceate a model
        problem = cvx.Problem(objective, constraints)
        # solve and focus on numeric problem
        problem.solve(solver=cvx.CLARABEL)
        # get solutions
        proj = p.value
        # normalize
        proj = proj / np.linalg.norm(proj)
        return torch.FloatTensor(proj)

In [36]:
# init model
reg = LinearRegression()

In [37]:
# init loss
ca_cos = coneAlignedCosine(optmodel)

In [38]:
lr = 5e-3
num_epochs = 1
log_step = 1
loss_log, regret_log = train(reg, ca_cos, lr, num_epochs, log_step)

Epoch   0, Loss:  -0.9936, Regret: 14.7692%
Elapsed Time: 7.93 Sec


##